<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install --upgrade pip
!pip install pyLDAvis
# import pyLDAvis
import pyLDAvis.gensim_models # don't skip this

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa


from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

In [91]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(5)

In [ ]:
bug_report["Priority"].value_counts()

In [ ]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

In [94]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 479 ms, sys: 2.86 ms, total: 482 ms
Wall time: 486 ms


In [ ]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

In [ ]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

In [ ]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

In [ ]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(5)

In [ ]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


In [ ]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

CPU times: user 17 s, sys: 445 ms, total: 17.4 s
Wall time: 24.7 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.092145  0.145782       1        1  17.455179
0     -0.194897  0.016708       2        1  13.628059
9     -0.067260  0.010966       3        1  13.166511
6     -0.148043  0.082484       4        1  12.000245
1     -0.163450 -0.067715       5        1  10.545057
3      0.181220  0.089899       6        1   7.875766
7      0.028291 -0.361849       7        1   6.998116
5      0.288532 -0.087651       8        1   6.386552
8     -0.074654  0.011786       9        1   6.250339
4      0.242407  0.159590      10        1   5.694176, topic_info=                  Term          Freq         Total Category  logprob  loglift
32             project  22671.000000  22671.000000  Default  30.0000  30.0000
372             source  15186.000000  15186.000000  Default  29.0000  29.0000
116             method  18870.000000  18870.000000  Default  28.0000  28.0000
13              editor  13331.000000  13331.000000  Default  27.0000  27.0000
589            message  12646.000000  12646.000000  Default  26.0000  26.0000
360             public   9576.000000   9576.000000  Default  25.0000  25.0000
761             import  10465.000000  10465.000000  Default  24.0000  24.0000
1341            thread   7775.000000   7775.000000  Default  23.0000  23.0000
109             select  14467.000000  14467.000000  Default  22.0000  22.0000
484             bundle   6385.000000   6385.000000  Default  21.0000  21.0000
52             display  10451.000000  10451.000000  Default  20.0000  20.0000
608             search   5405.000000   5405.000000  Default  19.0000  19.0000
103            eclipse  28561.000000  28561.000000  Default  18.0000  18.0000
358             plugin   8424.000000   8424.000000  Default  17.0000  17.0000
806          exception   6092.000000   6092.000000  Default  16.0000  16.0000
365             button   8876.000000   8876.000000  Default  15.0000  15.0000
23                miss   4047.000000   4047.000000  Default  14.0000  14.0000
448             return   7971.000000   7971.000000  Default  13.0000  13.0000
552               warn   4067.000000   4067.000000  Default  12.0000  12.0000
130            feature   6261.000000   6261.000000  Default  11.0000  11.0000
101             dialog  10789.000000  10789.000000  Default  10.0000  10.0000
626        perspective   4971.000000   4971.000000  Default   9.0000   9.0000
179             target   3594.000000   3594.000000  Default   8.0000   8.0000
4365        javathread   3389.000000   3389.000000  Default   7.0000   7.0000
4346       libjvmdylib   3382.000000   3382.000000  Default   6.0000   6.0000
3241            daemon   3333.000000   3333.000000  Default   5.0000   5.0000
698            browser   3178.000000   3178.000000  Default   4.0000   4.0000
361            resolve   3075.000000   3075.000000  Default   3.0000   3.0000
609             window   7029.000000   7029.000000  Default   2.0000   2.0000
487             string   5585.000000   5585.000000  Default   1.0000   1.0000
974             config    697.014252    697.905963   Topic1  -6.0931   1.7443
4243           equinox    695.660677    696.552492   Topic1  -6.0951   1.7443
2361            master    617.910821    618.802785   Topic1  -6.2136   1.7441
1174          download   2110.454225   2113.620655   Topic1  -4.9853   1.7440
346     configurations    562.158029    563.049765   Topic1  -6.3082   1.7439
3759         useragent    521.803015    522.696574   Topic1  -6.3826   1.7438
1242            releng    502.329207    503.220951   Topic1  -6.4207   1.7438
2871          baseline    407.378759    408.270495   Topic1  -6.6302   1.7433
3908         qualifier    249.857918    250.749673   Topic1  -7.1190   1.7420
3357         milestone    247.275070    248.166807   Topic1  -7.1294   1.7419
2052       annotations    237.044193    237.936131   Topic1  -7.1717   1.7418
4527           android    233.186

In [ ]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."


bow_vector = topic=lda_model[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
%%time


for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))


In [105]:
%%time

for c in range(10):
  exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))

CPU times: user 344 ms, sys: 203 ms, total: 547 ms
Wall time: 549 ms


In [106]:
%%time

for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x = topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

CPU times: user 1.53 s, sys: 9.59 ms, total: 1.54 s
Wall time: 1.54 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))
  exec('print(type(sent_{}))'.format(sent))

In [ ]:
!pip install glove_python

In [104]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

In [ ]:
# https://code.google.com/archive/p/word2vec/
# https://radimrehurek.com/gensim/models/word2vec.html
# https://radimrehurek.com/gensim/models/fasttext.html

In [ ]:
# The word2vec tool takes a text corpus as input and produces the word vectors as output
# A word vector is an attempt to mathematically represent the meaning of a word.

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))    # Path to a corpus file
  vector_size = 100                           # Dimensionality of the word vectors
  w = 6                                       #Maximum distance between the current and predicted word within a sentence
  min_count = 5                               # Ignores all words with total frequency lower than this.
  CBoW = 0                                    # Training algorithm: 1 for skip-gram; otherwise CBOW(The model tries to predict the target word by trying to understand the context of the surrounding words.).
  iterations = 100

  exec('w2vmodel{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  exec('path = get_tmpfile("word2vec{}.model")'.format(cluster))
  exec('w2vmodel{}.save("word2vec{}.model")'.format(cluster, cluster))

In [ ]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("ftmodel{}.model")'.format(cluster))
  exec('ftmodel{}.save("ftmodel{}.model")'.format(cluster, cluster))

In [110]:
%%time

for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(topic_{})'.format(cluster, cluster))
  exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("glove{}.model")'.format(cluster))
  exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

TypeError: ignored